# Day 1

## Part 1
Figure out the most calories carried by any elf.

In [1]:
; TODO stream process line by by line rather than read in the entire file into memory
(let ((lines (uiop:read-file-lines "input_day1.txt"))
      ; temporary accumulator for each elfs calories
      (cal_ct 0)
      ; variable to track the maximum calories per elf
      (max_cal 0))
    (dolist (line lines)
        ; elves are demarcated by empty lines
        (if (string= "" line)
            (progn (when (> cal_ct max_cal) (setq max_cal cal_ct)) (setq cal_ct 0))
            (incf cal_ct (parse-integer line))
        )
    )
 ; return the maximum number of calories found on an elf
 max_cal)
    

68923

## Part 2

Find the sum of the calories for the top 3 elves. 

In [2]:
(let (
      (lines (uiop:read-file-lines "input_day1.txt"))
      (cal_ct 0)
      (calories nil))
    (dolist (line lines)
        (if (string= "" line)
            (progn (push cal_ct calories) (setq cal_ct 0))
            (progn (incf cal_ct (parse-integer line)))
        )
    )
    (let ((sorted (sort calories #'>)))
         (+ (first sorted) (second sorted) (third sorted)))
)


200044

# Day 2

## Part 1

Score games of rock-paper-scissors. We'll try first with a brute force if-else chain.

In [3]:
(defun score_selected_shape (b)
     (if (string= b "X")
         ; 1 point for playing rock
         1
         (if (string= b "Y")
             ; 2 points for playing paper
             2
             (if (string= b "Z")
                 ; 3 points for playing scissors
                 3))))

(defun score_rock_paper_scissors (a b)
    ; they played rock
    (if (string= a "A")
        ; we played rock -> tie
        (if (string= b "X")
            3
            ; we played paper -> we won
            (if (string= b "Y")
                6
                ; we played scissors -> we lost
                (if (string= b "Z")
                    0)))
    ; they played paper
    (if (string= a "B")
        ; we played rock -> we lose
        (if (string= b "X")
            0
            ; we played paper -> tie
            (if (string= b "Y")
                3
                ; we played scissors -> we win
                (if (string= b "Z")
                    6)))
    ; they played scissors
    (if (string= a "C")
        ; we played rock -> we win
        (if (string= b "X")
            6
            ; we played paper -> we lose
            (if (string= b "Y")
                0
                ; we played scissors -> tie
                (if (string= b "Z")
                    3)))))))

(defun score_game (a b)
    "Score a single game of rock-paper-scissors. Opponent plays `a` from {A,B,C}` we play `b` from {X,Y,Z}"
    (+ (score_selected_shape b) (score_rock_paper_scissors a b)))
    
(let ((lines (uiop:read-file-lines "input_day2.txt")))
     (loop for line in lines sum (score_game (string (char line 0)) (string (char line 2)))))

SCORE_SELECTED_SHAPE

SCORE_ROCK_PAPER_SCISSORS

SCORE_GAME

11475

Now let's try association lists.

In [4]:
(let ((lines (uiop:read-file-lines "input_day2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3))))
    (loop for line in lines
          for tmp_shape_score = (cdr (assoc (char line 2) shape_score :test #'string=))
          for tmp_game_score = (cdr (assoc line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))

11475

## Part 2

Score games of rock paper scissors again.

In [5]:
(let ((lines (uiop:read-file-lines "input_day2.txt"))
      (shape_score '(("X" . 1)("Y" . 2) ("Z" . 3)))
      (game_score '(("A X" . 3)
                    ("A Y" . 6)
                    ("A Z" . 0)
                    ("B X" . 0)
                    ("B Y" . 3)
                    ("B Z" . 6)
                    ("C X" . 6)
                    ("C Y" . 0)
                    ("C Z" . 3)))
      (what_play  '(("A X" . "Z") ;; they chose rock, we need to lose so choose scissors
                    ("A Y" . "X")
                    ("A Z" . "Y")
                    ("B X" . "X") ;; they chose paper, we need to lose so choose rock
                    ("B Y" . "Y")
                    ("B Z" . "Z")
                    ("C X" . "Y") ;; they chose scissors, we need to lose so choose paper
                    ("C Y" . "Z")
                    ("C Z" . "X"))))
    (loop for line in lines
          for played = (cdr (assoc line what_play :test #'string=))
          for tmp_shape_score = (cdr (assoc played shape_score :test #'string=))
          for new_line = (replace line played :start1 2)
          for tmp_game_score = (cdr (assoc new_line game_score :test #'string=))
          sum (+ tmp_shape_score tmp_game_score)))


16862

# Day 3

## Part 1

Sum of priorities of items in both compartments.

In [6]:
;; alist mapping characters to integer priority
(defparameter priorities (concatenate 'list
                   (loop for i from 1 to 26 collect (cons (code-char (+ 96 i)) i))
                   (loop for i from 1 to 26 collect (cons (code-char (+ 64 i)) (+ 26 i)))))

(defun find_item_in_both (contents)
    "Find character that is in both the first half and second half of the `contents` string."
    (let ((first_half (subseq contents 0 (/ (length contents) 2)))
          (second_half (subseq contents (/ (length contents) 2))))
         ;; assume there is only one so can take first item in intersection of characters
         (first (intersection (coerce first_half 'list) (coerce second_half 'list))))
)

(let ((lines (uiop:read-file-lines "input_day3.txt")))
     (loop for line in lines
           sum (cdr (assoc (find_item_in_both line) priorities)))
)

PRIORITIES

FIND_ITEM_IN_BOTH

7903

In [7]:
(let ((lines (uiop:read-file-lines "input_day3.txt")))
    ;; step by groups of 3
     (loop for x upto (- (/ (length lines) 3) 1)
           for group = (subseq lines (* 3 x) (+ (* 3 x) 3))
           ;; do (print (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))))
           sum (cdr (assoc (first (reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) group))) priorities)))
)

2548

`intersection` assumes set lists where "A set doesn’t contain twice the same element and is unordered". ([cookbook intersection of lists](https://lispcookbook.github.io/cl-cookbook/data-structures.html#intersection-of-lists)). If we have repeated elements in the list then we get multiple results


In [8]:
(reduce #'intersection (mapcar (lambda (x) (coerce x 'list)) (list "hlLwDGLNnTrbNdtbdb" "ZfsvJbsJTrsttHTR" "vjPQJPFcmZjVFZFvbbVcJZJPBBhLMnDBznnGMDLMwBhlmBBl")))

(#\b #\b #\b)

# Day 4

## Part 1

Calculate number of complete overlaps.

In [9]:
(ql:quickload "cl-ppcre")

To load "cl-ppcre":
  Load 1 ASDF system:
    cl-ppcre
; Loading "cl-ppcre"
..

("cl-ppcre")

In [10]:
(let ((lines (uiop:read-file-lines "input_day4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; assumes ordered a,b and c,d
            (or (and (>= a c) (<= b d)) (and (>= c a) (<= d b))))
    )
)

588

In [11]:
(let ((lines (uiop:read-file-lines "input_day4.txt")))
     (loop for line in lines
           count (ppcre:register-groups-bind ((#'parse-integer a b c d))
            ("(\\d+)-(\\d+),(\\d+)-(\\d+)" line)
            ;; check whether any limit lies within the other bounds
            (or (and (>= a c) (<= a d)) (and (>= b c) (<= b d)) (and (>= c a) (<= c b)) (and (>= d a) (<= d b))))
    )
)

911

# Day 5

## Part 1

Rearange crates. Puzzle input:

```
    [H]         [D]     [P]        
[W] [B]         [C] [Z] [D]        
[T] [J]     [T] [J] [D] [J]        
[H] [Z]     [H] [H] [W] [S]     [M]
[P] [F] [R] [P] [Z] [F] [W]     [F]
[J] [V] [T] [N] [F] [G] [Z] [S] [S]
[C] [R] [P] [S] [V] [M] [V] [D] [Z]
[F] [G] [H] [Z] [N] [P] [M] [N] [D]
 1   2   3   4   5   6   7   8   9 
```

In [12]:
(defparameter crates (list
                      (list "W" "T" "H" "P" "J" "C" "F" )
                      (list "H" "B" "J" "Z" "F" "V" "R" "G")
                      (list "R" "T" "P" "H")
                      (list "T" "H" "P" "N" "S" "Z")
                      (list "D" "C" "J" "H" "Z" "F" "V" "N")
                      (list "Z" "D" "W" "F" "G" "M" "P")
                      (list "P" "D" "J" "S" "W" "Z" "V" "M")
                      (list "S" "D" "N")
                      (list "M" "F" "S" "Z" "D")))

(let ((lines (uiop:read-file-lines "input_day5.txt")))
     (loop for line in lines
           do (ppcre:register-groups-bind ((#'parse-integer to_move from_stack to_stack))
            ("move (\\d+) from (\\d+) to (\\d+)" line)
            (let ((tmp_stack nil))
                 (progn
                     (loop repeat to_move do (push (pop (elt crates (1- from_stack))) tmp_stack))
                     (loop repeat to_move do (push (pop tmp_stack) (elt crates (1- to_stack))))
                )))))
(loop for stack in crates do (format t "~a" (first stack)))

CRATES

NIL

NIL

ZFSJBPRFP